In [ ]:

decision_mode = '''Given the company's environment, the most appropriate production planning models from the book "Factory Physics" would be:

1. **Aggregate Planning (AP), Product Mix Planning II. Basic model**: This model is suitable because the company deals with one main product and multiple minor products. The basic product mix planning model will help in managing the production of multiple products explicitly.

2. **Aggregate Planning (AP), Product Mix Planning III. Capacitated resources**: Since the capacity of workstations, plant size, and labor force size are constraining factors, this model will help in managing these constraints effectively.

3. **Aggregate Planning (AP), Product Mix Planning VI. Overtime**: Given that the company uses overtime, this model will help in planning and optimizing the use of overtime to meet production demands.

4. **Workforce Planning (WP), VIII. Basic model**: This model is essential because the company has a mix of long-term and short-term contracts and uses overtime. Workforce planning will help in managing the labor force size and scheduling overtime efficiently.

These models combined will provide a comprehensive approach to managing the production planning, considering the constraints and variations in the company's environment.'''

Input_Data= '''
time_horizon = 6
products_total = 4
workstations_total = 4
ressources_total = 4
min_sales = [
    [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
    [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],
    [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],
    [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],
]
max_demand = [
    [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],
    [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],
    [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],
    [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],
]
capacity = [
    [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],
    [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],
    [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],
    [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],
]
production_time = [
    [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],
    [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],
    [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],
    [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],
]
resource_requirement = [
    [0.8, 0.5, 1.0, 4.0],
    [0.5, 1.3, 3.0, 1.0],
    [2.0, 1.0, 2.0, 4.0],
    [2.0, 6.0, 4.0, 2.0],
]

initial_inventory = [41.0, 58.0, 31.0, 38.0]


back_orders_penalty = [5, 6, 7, 2]

Specific profits and holding costs for each product
profit_per_unit = [132.0, 83.0, 225.0, 131.0]
holding_cost_per_unit = [3.0, 8.0, 5.0, 1.0]

worker_hours_per_product = 2.0
hourly_wage = 20.0
hourly_wage_overtime = 30.0
hiring_cost = 10.0
layoff_cost = 50.0

resource_requirement = [
    [300.8, 30.5, 1.0, 3.0],
    [5.5, 14.3, 33.0, 10.0],
    [50.0, 14.0, 2.0, 40.0],
    [2.0, 6.0, 40.0, 1.0],
]

resource_capacity = [
    [100.0, 8345.0, 1417.0, 407.0, 20.0, 290.0],
    [25.0, 41.0, 338.0, 210.0, 405.0, 2090.0],
    [7103.0, 30.0, 2007.0, 430.0, 90.0, 140.0],
    [200.0, 3001.0, 470.0, 2109.0, 140.0, 203.0],
]


'''

# Defining some functions


def extract_python_code_manual(text):
    """
    Manually extracts Python code enclosed between ```python and ``` markers.

    Parameters:
    - text (str): The input text containing the Python code blocks.

    Returns:
    - list of str: A list containing all extracted Python code blocks.
    """
    code_blocks = []
    capture = False
    current_block = []

    # Split the text into lines for line-by-line processing
    lines = text.split('\n')

    for line in lines:
        # Check if the line contains the start marker
        if line.strip() == '```python':
            capture = True
            current_block = []  # Start a new block
        elif line.strip() == '```' and capture:
            # If we find the end marker and are in capture mode, stop capturing
            capture = False
            # Add the current block to the list of code blocks
            code_blocks.append('\n'.join(current_block))
        elif capture:
            # If we are in capture mode, add the line to the current block
            current_block.append(line)

    return code_blocks


def compare_total_profits(input_str1, input_str2, previous_solution, secondary_solution, result_prev_fb, result_second_fb):
    # Regular expression to find "Total Profit" followed by any whitespace and a number
    pattern = r"Total Profits across all products and periods:\s*(\d+\.\d+)"

    # Search for the pattern in both strings
    match1 = re.search(pattern, input_str1)
    match2 = re.search(pattern, input_str2)

    # Extract the profit values if found, otherwise default to 0
    profit1 = float(match1.group(1)) if match1 else 0
    profit2 = float(match2.group(1)) if match2 else 0

    # Determine the higher profit and corresponding input string
    if profit1 >= profit2:
        winner_profit = profit1
        winner_input = previous_solution
        winner_fb = result_prev_fb
    else:
        winner_profit = profit2
        winner_input = secondary_solution
        winner_fb = result_second_fb

    # Return the higher profit value and the corresponding input string
    return winner_profit, winner_input, winner_fb

In [ ]:
import numpy as np
import simpy
import random

def run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, seed=42):
    random.seed(seed)
    np.random.seed(seed)

    num_periods = 6
    back_orders_penalty = [5, 6, 7, 2]
    initial_inventory = [41.0, 58.0, 31.0, 38.0]

    # Specific profits and holding costs for each product
    profit_per_unit = [132.0, 83.0, 225.0, 131.0]
    holding_cost_per_unit = [3.0, 8.0, 5.0, 1.0]

    worker_hours_per_product = 2.0  # Assumption
    hourly_wage = 20.0  # Assumption
    hourly_wage_overtime = 35.0  # Assumption
    hiring_cost = 10.0  # Assumption
    layoff_cost = 5.0  # Assumption
    initial_workers = 10  # Assumption

    production_time = [
        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],
        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],
        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],
        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],
    ]

    yield_loss = [
        [0.9, 0.7, 0.9, 0.8],
        [0.8, 0.6, 0.95, 0.8],
        [0.9, 0.75, 0.9, 0.78],
        [0.95, 0.87, 0.97, 0.98],
    ]

    capacity_reduction_factor = 0.9
    capacity = [
        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],
        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],
        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],
        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],
    ]

    max_demand = [
        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],
        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],
        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],
        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],
    ]

    min_sales = [
        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],
        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],
        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],
    ]

    Workstation_capacities = [25, 75, 30]

    product_workflows = [
        [0, 1],  # Product 0
        [1, 2],  # Product 1
        [0],     # Product 2
        [1, 2],  # Product 3
    ]

    env = simpy.Environment()

    class Workstation:
        def __init__(self, env, capacity, id):
            self.env = env
            self.capacity = capacity
            self.id = id
            self.queue = simpy.Resource(env, capacity=capacity)
            self.working = True

        def fail(self, duration):
            self.working = False
            yield self.env.timeout(duration)
            self.working = True

        def process(self, produced):
            if self.working:
                with self.queue.request() as request:
                    yield request
                    processing_time = produced / self.capacity
                    yield self.env.timeout(processing_time)
            else:
                yield self.env.timeout(1)  # Wait for 1 unit of time before trying again

    class WorkerShift:
        def __init__(self, env, shifts):
            self.env = env
            self.shifts = shifts
            self.current_shift = 0
            self.current_workers = shifts[0]['workers']

        def change_shift(self):
            while True:
                yield self.env.timeout(8)  # Change shift every 8 hours
                self.current_shift = (self.current_shift + 1) % len(self.shifts)
                self.current_workers = self.shifts[self.current_shift]['workers']
                print(f"Shift change at time {self.env.now:.2f}. Current workers: {self.current_workers}")

    class MultiProductFactory:
        def __init__(self, env, workstations, worker_shift):
            self.env = env
            self.inventory = list(initial_inventory)
            self.total_profits = 0.0
            self.period_results = []
            self.workstations = workstations
            self.worker_shift = worker_shift
            self.resources = [list(cap) for cap in capacity]  # Initialize resource capacities

        def process_at_workstation(self, workstation, produced, product_id):
            with workstation.queue.request() as request:
                yield request
                processing_time = produced / workstation.capacity
                yield self.env.timeout(processing_time)

        def run_factory(self):
            for period in range(num_periods):
                print(f"\n--- Period {period + 1} ---")
                period_data = {'Period': period, 'Products': []}
                total_produced = 0
                total_sales = 0

                # Simulate workstation outages randomly
                for ws in self.workstations:
                    if ws.id == 2:  # Example: Specific workstation id for potential outage
                        if np.random.poisson(1/5) >= 1:  # Poisson distribution to simulate outages
                            downtime_duration = 2  # Downtime duration in time units
                            print(f"Workstation {ws.id} starting downtime at time {self.env.now} for {downtime_duration} periods.")
                            yield self.env.process(ws.fail(downtime_duration))
                            print(f"Workstation {ws.id} ending downtime at time {self.env.now}.")

                for product_id in range(len(self.inventory)):
                    produced = produced_Prev[product_id][period]

                    # Resource requirements and checking capacities
                    for workstation_id in product_workflows[product_id]:
                        workstation = self.workstations[workstation_id]
                        required_resource = produced * production_time[workstation_id][period] / yield_loss[workstation_id][product_id]
                        if required_resource > self.resources[workstation_id][period]:
                            produced = self.resources[workstation_id][period] * yield_loss[workstation_id][product_id] / production_time[workstation_id][period]
                            self.resources[workstation_id][period] = 0
                        else:
                            self.resources[workstation_id][period] -= required_resource

                    self.inventory[product_id] += produced
                    total_produced += produced

                    if produced > 0:
                        yield self.env.process(self.process_at_workstation(workstation, produced, product_id))

                    fluctuation = random.uniform(-0.1, 0.1)
                    actual_demand = sale_Prev[product_id][period] * (1 + fluctuation)
                    actual_demand = min(actual_demand, max_demand[product_id][period])
                    sold = max(min(self.inventory[product_id], actual_demand), min_sales[product_id][period])
                    missed_sales = max(actual_demand - sold, 0)

                    revenue = sold * profit_per_unit[product_id]
                    holding_costs = self.inventory[product_id] * holding_cost_per_unit[product_id]
                    period_profit = revenue - holding_costs
                    self.total_profits += period_profit

                    self.inventory[product_id] = max(self.inventory[product_id] - sold, 0)

                    product_data = {
                        'Product ID': product_id,
                        'Produced': produced,
                        'Sold': sold,
                        'Missed Sales': missed_sales,
                        'Ending Inventory': self.inventory[product_id],
                        'Period Profit': period_profit
                    }
                    period_data['Products'].append(product_data)

                total_required_worker_hours = total_produced * worker_hours_per_product
                overtime_hours = max(total_required_worker_hours - self.worker_shift.current_workers * 8, 0)
                regular_hours = min(total_required_worker_hours, self.worker_shift.current_workers * 8)
                hiring_needed = max((total_required_worker_hours - regular_hours - overtime_hours) / 8, 0)
                layoffs_needed = max((self.worker_shift.current_workers * 8 - total_required_worker_hours) / 8, 0)

                self.worker_shift.current_workers += hiring_needed - layoffs_needed

                # Calculate workforce utilization
                available_worker_hours = self.worker_shift.current_workers * 8  # Assuming each worker works for 8 hours per shift
                workforce_utilization = total_required_worker_hours / available_worker_hours if available_worker_hours > 0 else 0

                overtime_cost = overtime_hours * hourly_wage_overtime
                regular_cost = regular_hours * hourly_wage
                hiring_cost_total = hiring_needed * hiring_cost
                layoff_cost_total = layoffs_needed * layoff_cost

                period_profit -= (overtime_cost + regular_cost + hiring_cost_total + layoff_cost_total)
                self.total_profits += period_profit

                period_data.update({
                    'Overtime Hours': overtime_hours,
                    'Regular Hours': regular_hours,
                    'Hired': hiring_needed,
                    'Fired': layoffs_needed,
                    'Workforce': self.worker_shift.current_workers,
                    'Workforce Utilization': workforce_utilization
                })

                self.period_results.append(period_data)
                yield self.env.timeout(1)

    produced_Prev = np.array(produced_Prev)
    sale_Prev = np.array(sale_Prev)
    min_sales = np.array(min_sales)

    workstations = [Workstation(env, capacity, id) for id, capacity in enumerate(Workstation_capacities)]
    worker_shift = WorkerShift(env, shifts=[
        {'start': 0, 'end': 8, 'workers': 223},
        {'start': 8, 'end': 16, 'workers': 230},
        {'start': 16, 'end': 24, 'workers': 55}
    ])
    factory = MultiProductFactory(env, workstations, worker_shift)

    env.process(worker_shift.change_shift())
    env.process(factory.run_factory())
    env.run(until=num_periods * 24)  # Ensure the environment runs through all periods

    return factory.period_results, f"\nTotal Profits across all products and periods: {factory.total_profits:.2f}"


In [ ]:
explain_example = """ # Table: Implemented Strategies
# | Strategy | Description | Reason for Implementation |
# | --- | --- | --- |
# | [TEXT] | [TEXT] | [TEXT] |
"""

In [ ]:
# Define the filename
filename = "8 Code Pattern.txt"

# Read the content of the file
with open(filename, 'r') as file:
    RAG = file.read()

# Print the content
print(RAG)

type(RAG)

Function to optimize aggregate procution planning with a single product.

Parameters:
t_bar (int): Time horizon.
revenue (float): Revenue per unit.
holding_costs (float): Holding costs per unit.
max_demand (dict): Demand for each period.
capacity (dict): Capacity for each period.
initial_inventory (float): Initial inventory at the start of the period.

Returns:
dict: Optimal values of decision variables and total profit.
"""
from pulp import LpMaximize, LpProblem, LpVariable, lpSum
def optimize_production(time_horizon, profit, holding_costs, max_demand, capacity, initial_inventory):


    # Index ranges for products, workstations, and periods
    # periods = range(1, bar_t + 1)

    # Define the problem
    problem = LpProblem("Maximize_Profit", LpMaximize)

    # Create decision variables
    Amount_produced = [LpVariable(f"Amount_produced_period:{t}", lowBound=0) for t in range(time_horizon)]
    Amount_sold = [LpVariable(f"Amount_sold_period:{t}", lowBound=0) for t in range(time_hor

str

In [ ]:
!pip install simpy scipy

In [ ]:
import subprocess
import sys

def install_simpy():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "simpy"])

def install_numpy():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy"])

def install_pulp():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pulp"])

def install_openai():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "openai"])

# Function to install all packages
def install_all_packages():
    install_simpy()
    install_numpy()
    install_pulp()
    install_openai()

# Install all packages
install_all_packages()


In [ ]:
print('printing works...')

printing works...


In [ ]:
import simpy
import random
import numpy as np
import io
import sys
import ast
import re
import pulp
import random
import os
import scipy

In [ ]:
!pip install openai==0.28

import os
import openai

openai.api_key = ''
os.environ['OPENAI_API_KEY'] = openai.api_key

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.37.1
    Uninstalling openai-1.37.1:
      Successfully uninstalled openai-1.37.1


In [ ]:
Num_Data = Input_Data

# Construct the chat history with the current data and previous solution
chat_history = [
{"role": "system", "content": "You are a helpful assistant."},
{"role": "user", "content": f"Print out the Python code for the following mathematical problem: {decision_made}. Incorporate all variable costs into the python function of the mathematic model. This shall serve as a 8 code patterns pattern: {RAG}. Use one code pattern or a blend of multiple code patterns. Use as the follow data as real life input data and include it in the code outputted: {Num_Data}. Futher instruction: Only print executable code in Python, noting above and below, because I immediately want to execute it using the eval. Write the code in a way that when running the code, the dicision variables will be outputted as (multidimensional) arrays of naked numbers. Also return the profit earned. Therefore, the input data (and its dimensions) you choose to include into the code and the mathemtaic model must match the optimization model. For the decision varaibles amount produced and sales and stock use this exact terminology as printouts: Produced: XXX, Sale: XXX, Stock: XXX. In case the LP you create includes workforce planning, also include: Workforce: XXX, Hires: XXX, Firings: XXX, Overtime: XXX"}
]

response = openai.ChatCompletion.create(model="gpt-4o", messages=chat_history)
print(response.choices[0].message.content)

# Save the alternative solution in the secondary_solutions list
previous_solution = response.choices[0].message.content

# At this point, secondary_solutions contains the new solutions corresponding to each item in data_list

NameError: name 'RAG' is not defined

In [ ]:
printouts = """
    "Produced": [[Amount_produced[i][t].varValue for t in range(time_horizon)] for i in range(products_total)],
    "Sale": [[Amount_sold[i][t].varValue for t in range(time_horizon)] for i in range(products_total)],
    "Stock": [[Inventory[i][t].varValue for t in range(time_horizon)] for i in range(products_total)],
    "Workforce": [Workforce[t].varValue for t in range(time_horizon)],
    "Hires": [Hires[t].varValue for t in range(time_horizon)],
    "Firings": [Firings[t].varValue for t in range(time_horizon)],
    "Overtime": [Overtime[t].varValue for t in range(time_horizon)]"""

In [ ]:
previous_solution = iterative_MINLP

In [ ]:
previous_solution = """```python
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Define the function
def optimize_production():
    # Input data
    time_horizon = 6
    products_total = 4
    workstations_total = 4
    ressources_total = 4

    profit = [132.0, 813.0, 225.0, 131.0]
    holding_costs = [13.0, 18.0, 15.0, 13.0]

    min_sales = [
        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],
        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],
        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],
    ]

    max_demand = [
        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],
        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],
        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],
        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],
    ]

    capacity = [
        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],
        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],
        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],
        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],
    ]

    production_time = [
        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],
        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],
        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],
        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],
    ]

    initial_inventory = [41.0, 58.0, 31.0, 38.0]
    yield_loss = [
        [0.9, 0.7, 0.9, 0.8],
        [0.8, 0.6, 0.95, 0.8],
        [0.9, 0.75, 0.9, 0.78],
        [0.95, 0.87, 0.97, 0.98],
    ]
    capacity_reduction_factor = 0.9

    # New data required for the function
    worker_hours_per_product = 2.0  # Assumption
    hourly_wage = 20.0  # Assumption
    hourly_wage_overtime = 35.0  # Assumption
    hiring_cost = 10.0  # Assumption
    layoff_cost = 5.0  # Assumption
    initial_workers = 10  # Assumption

    # Create the problem
    problem = LpProblem("Advanced_Supply_Chain_Optimization", LpMaximize)

    # Index ranges for products, workstations, and periods
    products = range(products_total)
    workstations = range(workstations_total)
    periods = range(time_horizon)

    # Decision Variables
    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]
    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]
    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]
    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]
    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]
    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]
    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]

    # Objective Function
    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)
    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)
    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)
    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)
    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)
    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)
    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term

    # Constraints
    for i in products:
        for t in periods:
            # Sales constraints
            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"
            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"
            # Inventory balance constraint
            if t == 0:
                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"
            else:
                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"

    for j in workstations:
        for t in periods:
            # Adjusted capacity constraint
            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <= \
                       capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"

    for t in periods:
        # Workforce constraints
        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"
        if t == 0:
            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"
        else:
            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"

    # Solve the problem
    problem.solve()

    # Output the results
    print_result = lambda s, v: [f"{s}: {[[v[i][t].varValue for t in periods] for i in products]}" if isinstance(v[0], list) else f"{s}: {[v[t].varValue for t in periods]}" for v in [v]]

    # Extract the optimization results
    results = {
        "Produced": [[Produced[i][t].varValue for t in periods] for i in products],
        "Sale": [[Sale[i][t].varValue for t in periods] for i in products],
        "Stock": [[Stock[i][t].varValue for t in periods] for i in products],
        "Workforce": [Workforce[t].varValue for t in periods],
        "Hires": [Hires[t].varValue for t in periods],
        "Firings": [Firings[t].varValue for t in periods],
        "Overtime": [Overtime[t].varValue for t in periods],
        "Profit": pulp.value(problem.objective)
    }

    for key, value in results.items():
        print(f"{key}: {value}")

    return results

# Call the function to optimize production
optimize_production()
```"""


In [ ]:
import random
seed = 42

py_code_p = extract_python_code_manual(previous_solution)
# Redirect stdout to capture prints
old_stdout = sys.stdout  # Save the old state
new_stdout = io.StringIO()
sys.stdout = new_stdout
# Execute the code
exec(py_code_p[0]) # do not intililize the inventory here!!!!

# Restore stdout to its original state
sys.stdout = old_stdout
# Get the output and work with it
execp = new_stdout.getvalue()




In [ ]:
#patterns = {
#    "Produced": r"Produced: (\[\[.*?\]\])",
#    "Sale": r"Sale: (\[\[.*?\]\])",
#    "Stock": r"Stock: (\[\[.*?\]\])"
#}

patterns = {
    "Produced": r"Produced: (\[\[.*?\]\])",
    "Sale": r"Sale: (\[\[.*?\]\])",
    "Stock": r"Stock: (\[\[.*?\]\])",
    "Overtime": r"Overtime: (\[.*?\])",
    "Workforce": r"Workforce: (\[.*?\])",
    "Hires": r"Hires: (\[.*?\])",
    "Firings": r"Firings: (\[.*?\])",
    "Profit": r"Profit: (.*?)\n"
}

# Dictionary to hold the extracted matrices
matrices = {}

# Extract and evaluate each matrix
for key, pattern in patterns.items():
    match = re.search(pattern, execp, re.DOTALL)
    if match:
        matrices[key] = ast.literal_eval(match.group(1))

# Access the matrices
produced_Prev = matrices.get("Produced")
sale_Prev = matrices.get("Sale")
stock_Prev = matrices.get("Stock")

overtime_Prev = matrices.get("Overtime")
workers_Prev = matrices.get("Workforce")
hired_Prev = matrices.get("Hires")
fired_Prev = matrices.get("Firings")

In [ ]:
previous_feedback = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, seed = 42)[0]


--- Period 1 ---

--- Period 2 ---
Workstation 2 starting downtime at time 2.3 for 2 periods.
Workstation 2 ending downtime at time 4.3.
Shift change at time 8.00. Current workers: 230

--- Period 3 ---

--- Period 4 ---
Shift change at time 16.00. Current workers: 55
Shift change at time 24.00. Current workers: 223

--- Period 5 ---
Shift change at time 32.00. Current workers: 230

--- Period 6 ---
Shift change at time 40.00. Current workers: 55
Shift change at time 48.00. Current workers: 223
Shift change at time 56.00. Current workers: 230
Shift change at time 64.00. Current workers: 55
Shift change at time 72.00. Current workers: 223
Shift change at time 80.00. Current workers: 230
Shift change at time 88.00. Current workers: 55
Shift change at time 96.00. Current workers: 223
Shift change at time 104.00. Current workers: 230
Shift change at time 112.00. Current workers: 55
Shift change at time 120.00. Current workers: 223
Shift change at time 128.00. Current workers: 230
Shift ch

In [ ]:
previous_feedback

[{'Period': 0,
  'Products': [{'Product ID': 0,
    'Produced': 30.0,
    'Sold': 71.0,
    'Missed Sales': 1.9798605381019456,
    'Ending Inventory': 0.0,
    'Period Profit': 9159.0},
   {'Product ID': 1,
    'Produced': 0.0,
    'Sold': 30.770073135514135,
    'Missed Sales': 0.0,
    'Ending Inventory': 27.229926864485865,
    'Period Profit': 2089.9160702476734},
   {'Product ID': 2,
    'Produced': 0.0,
    'Sold': 23.0,
    'Missed Sales': 0,
    'Ending Inventory': 8.0,
    'Period Profit': 5020.0},
   {'Product ID': 3,
    'Produced': 27.0,
    'Sold': 20.0,
    'Missed Sales': 0,
    'Ending Inventory': 45.0,
    'Period Profit': 2555.0}],
  'Overtime Hours': 0,
  'Regular Hours': 114.0,
  'Hired': 0.0,
  'Fired': 208.75,
  'Workforce': 14.25,
  'Workforce Utilization': 1.0},
 {'Period': 1,
  'Products': [{'Product ID': 0,
    'Produced': 28.0,
    'Sold': 28.0,
    'Missed Sales': 1.3242387993184686,
    'Ending Inventory': 0.0,
    'Period Profit': 3612.0},
   {'Product ID

In [ ]:
previous_solution_bu = previous_solution
previous_feedback_bu = previous_feedback

In [ ]:
def cache_thing(name, thing):
    from datetime import datetime
    import os

    # Create the cache directory if it doesn't exist
    if not os.path.exists('cache'):
        os.makedirs('cache')
    with open(f'cache/{name}-{datetime.now()}.txt', 'w') as f:
        f.write(thing)

In [ ]:
import openai
import sys
import io
import re
import ast
import random

def chance(ps, pf, explain_example, number_of_executions = 2):
    previous_solution = ps
    previous_feedback = pf

    counter = 0
    seed = 42

    while counter < number_of_executions:
        chat_history = [
            {"role": "user", "content": f"""Print out the Python MINLP code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (python at the beginning of the code, at the end of the code).
            In the beginning of the code, don't forget the python in python, at the end of the code don't forget the !!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Also use Mixed-Integer Non-Linear Programming (MINLP) for objective function and constraints using scipy package which I installed already (numpy and pulp installed already too).
            Also use embedded interpretable functions. Whenever you modify the optimization model, include to say what the real life implications would be, how this modification would be interpreted in a real life manufacturing system. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}.
            To do this, find new storage strategies, production, and sales strategies (and workforce strategies if such elements appear in the current LP model) based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned and keep the strategies that exist already, and add one more.
            Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays.
            Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX and so on ..... (don't forget to print out any decision variable that was also used in the initial optimization model code.
            Print it out as a python list not numpy list.) If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function, or for workforce), description strategy, and reason for this: (= which problem(s) detected in the last performance feedback the newly added strategy is working against.
            Explicitly name the problem spotted in the simulation feedback!). Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example} (collecting the strategies in this table, not deleting the older ones). These are just examples and should show the data structure!
            Try out only one new additionally added extension/new strategy compared to the LP model that I've shown you in the beginning of this prompt. In case you already see modifications and its annotations in the first code chunk, just add one more. Modify the optimization model and make use of non-linear strategies! While making sure the model is executable as it is with pulp and scipy having been installed already.
            This can serve you as an example what I mean: {Schablone}. Don't necessarily copy the exact strategies, its serves to show you sth. about the MINLP formatting (btw, in this example that you should iterate on, initial workers at just and integer, not an array, unlike the MINLP formatting pattern I've shown you). It can show you how to create an executable code.
            Alter objective function or constraints in an LP, non-LP or mixed-integer way."""}
        ]

        attempt = 1
        gen_out = None
        while attempt <= 3:
            old_stdout = sys.stdout  # Save the old state
            try:
                response = openai.ChatCompletion.create(model="gpt-4o", messages=chat_history)
                print(response.choices[0].message.content)
                cache_thing('response', response.choices[0].message.content)

                prediction = response.choices[0].message.content
                secondary_solution = "".join(prediction)
                print(secondary_solution)

                # Extracting the python code
                py_code_s = extract_python_code_manual(secondary_solution)

                # Redirect stdout to capture prints
                new_stdout = io.StringIO()
                sys.stdout = new_stdout
                # Execute the code
                exec(py_code_s[0])

                # Get the output and work with it
                gen_out = new_stdout.getvalue()
                cache_thing('eval', gen_out)

                # Restore stdout to its original state
                sys.stdout = old_stdout

                attempt += 1
                break
            except Exception as e:
                print(f'Failed attempt {attempt} due to error: {str(e)}')
                error_feedback = str(e)
                chat_history.append({"role": "user", "content": f"The execution failed with the following error: {error_feedback}. Modify the previous solution to correct this error and make sure the new solution runs successfully."})
                attempt += 1
            finally:
              sys.stdout = old_stdout

        if gen_out is None:
            print(f"Attempt {attempt} failed three times. Proceeding to the next iteration.")
            counter += 1
            continue

        py_code_p = extract_python_code_manual(previous_solution)
        cache_thing('py_code_p', str(py_code_p))

        # Redirect stdout to capture prints
        old_stdout = sys.stdout  # Save the old state
        new_stdout = io.StringIO()
        sys.stdout = new_stdout

        # Execute the code
        exec(py_code_p[0])
        # Restore stdout to its original state
        sys.stdout = old_stdout
        # Get the output and work with it
        execp = new_stdout.getvalue()

        # Define regex patterns for each matrix
        patterns = {
            "Produced": r"Produced: (\[\[.*?\]\])",
            "Sale": r"Sale: (\[\[.*?\]\])",
            "Stock": r"Stock: (\[\[.*?\]\])",
            "Overtime": r"Overtime: (\[.*?\])",
            "Workforce": r"Workforce: (\[.*?\])",
            "Hires": r"Hires: (\[.*?\])",
            "Firings": r"Firings: (\[.*?\])",
            "Profit": r"Profit: (.*?)\n"
        }

        # Dictionary to hold the extracted matrices
        matrices = {}

        # Extract and evaluate each matrix
        for key, pattern in patterns.items():
            match = re.search(pattern, execp, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        cache_thing('execp', execp)
        cache_thing('prev_matrices', str({ 'matrices': str(matrices), 'seed': seed }))

        # Access the matrices
        produced_Prev = matrices.get("Produced")
        sale_Prev = matrices.get("Sale")
        stock_Prev = matrices.get("Stock")
        overtime_Prev = matrices.get("Overtime")
        workers_Prev = matrices.get("Workforce")
        hired_Prev = matrices.get("Hires")
        fired_Prev = matrices.get("Firings")

        # Define regex patterns for each matrix
        patterns = {
            "Produced": r"Produced: (\[\[.*?\]\])",
            "Sale": r"Sale: (\[\[.*?\]\])",
            "Stock": r"Stock: (\[\[.*?\]\])",
            "Overtime": r"Overtime: (\[.*?\])",
            "Workforce": r"Workforce: (\[.*?\])",
            "Hires": r"Hires: (\[.*?\])",
            "Firings": r"Firings: (\[.*?\])",
            "Profit": r"Profit: (.*?)\n"
        }

        # Dictionary to hold the extracted matrices
        matrices = {}

        # Extract and evaluate each matrix
        for key, pattern in patterns.items():
            match = re.search(pattern, gen_out, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        # Access the matrices
        produced_Second = matrices.get("Produced")
        sale_Second = matrices.get("Sale")
        stock_Second = matrices.get("Stock")
        overtime_Second = matrices.get("Overtime")
        workers_Second = matrices.get("Workforce")
        hired_Second = matrices.get("Hires")
        fired_Second = matrices.get("Firings")

        result_prev = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, seed)[1]
        result_second = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, seed)[1]

        result_prev_fb = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, seed)[0]
        result_second_fb = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, seed)[0]

        winner_profit, winner_input, winner_fb = compare_total_profits(result_prev, result_second, previous_solution, secondary_solution, result_prev_fb, result_second_fb)
        print("Higher Total Profit:", winner_profit)
        print("Winner Input:", winner_input)
        print("Winner Feedback:", winner_fb)

        previous_solution = winner_input
        previous_feedback = winner_fb

        print(f"Execution {counter + 1}")

        # Increment the counter at the end of each loop iteration
        counter += 1
        seed = seed+1

    return winner_input


In [ ]:
collect_array = []

number_of_iterations = 1
count = 0

while count < number_of_iterations:
    collect_array.append(chance(ps=previous_solution_bu, pf=previous_feedback_bu, explain_example=explain_example, number_of_executions=5))
    count += 1

print(collect_array)

Sure, I will make the modifications you requested by introducing a new strategy for dynamic inventory cost handling: implementing a quadratic holding cost function. The existing holding costs per unit time will be scaled non-linearly to better simulate real-world scenarios where holding more inventory becomes increasingly expensive.

Here's the updated solution in Python, with the requested changes:

```python
import pulp
import numpy as np
from scipy.optimize import minimize

# Input data
time_horizon = 6
products_total = 4
workstations_total = 4
ressources_total = 4

profit = [132.0, 813.0, 225.0, 131.0]
holding_costs = [13.0, 18.0, 15.0, 13.0]

min_sales = [
    [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
    [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],
    [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],
    [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],
]

max_demand = [
    [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],
    [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],
    [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],
   

In [ ]:
collect_array = []

number_of_iterations = 1
count = 0

while count < number_of_iterations:
    collect_array.append(chance(ps=previous_solution_bu, pf=previous_feedback_bu, explain_example=explain_example, number_of_executions=5))
    count += 1

print(collect_array)

KeyboardInterrupt: 

In [ ]:
Schablone = """Sure, I will make the modifications you requested: introducing a new strategy for limiting workforce hires and lay-offs within certain periods and using Mixed-Integer Non-Linear Programming (MINLP) with the scipy.optimize package.\n\nHere is the updated solution in Python, with the requested changes:\n\npython\nimport pulp\nimport numpy as np\nfrom scipy.optimize import minimize\n\n# Input data\ntime_horizon = 6\nproducts_total = 4\nworkstations_total = 4\nressources_total = 4\n\nmin_sales = [\n [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n]\nmax_demand = [\n [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n]\ncapacity = [\n [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],\n [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],\n]\nproduction_time = [\n [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],\n [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],\n [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],\n [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],\n]\nresource_requirement = [\n [300.8, 30.5, 1.0, 3.0],\n [5.5, 14.3, 33.0, 10.0],\n [50.0, 14.0, 2.0, 40.0],\n [2.0, 6.0, 40.0, 1.0],\n]\nresource_capacity = [\n [100.0, 8345.0, 1417.0, 407.0, 20.0, 290.0],\n [25.0, 41.0, 338.0, 210.0, 405.0, 2090.0],\n [7103.0, 30.0, 2007.0, 430.0, 90.0, 140.0],\n [200.0, 3001.0, 470.0, 2109.0, 140.0, 203.0],\n]\ninitial_inventory = [41.0, 58.0, 31.0, 38.0]\nback_orders_penalty = [5, 6, 7, 2]\nprofit_per_unit = [132.0, 83.0, 225.0, 131.0]\nholding_cost_per_unit = [3.0, 8.0, 5.0, 1.0]\n\nworker_hours_per_product = 2.0\nhourly_wage = 20.0\nhourly_wage_overtime = 30.0\nhiring_cost = 10.0\nlayoff_cost = 50.0\ninitial_workers = [50, 60, 70, 80]\n\ndef minlp_optimization_model(time_horizon, products_total, workstations_total, max_demand, min_sales, \n production_time, capacity, profit_per_unit, holding_cost_per_unit, initial_inventory, \n resource_requirement, resource_capacity, back_orders_penalty, \n worker_hours_per_product, hourly_wage, hourly_wage_overtime, hiring_cost, layoff_cost, initial_workers):\n\n # Create the problem\n problem = pulp.LpProblem("Advanced_Supply_Chain_Optimization_MINLP", pulp.LpMaximize)\n\n # Decision Variables\n Amount_produced = [[pulp.LpVariable(f"Produced_{i}{t}", lowBound=0, cat='Continuous') for t in range(time_horizon)] for i in range(products_total)]\n Amount_sold = [[pulp.LpVariable(f"Sale{i}{t}", lowBound=0, cat='Continuous') for t in range(time_horizon)] for i in range(products_total)]\n Inventory = [[pulp.LpVariable(f"Stock{i}{t}", lowBound=0, cat='Continuous') for t in range(time_horizon)] for i in range(products_total)]\n Workforce = [pulp.LpVariable(f"Workforce{t}", lowBound=0, cat='Continuous') for t in range(time_horizon)]\n Hires = [pulp.LpVariable(f"Hires_{t}", lowBound=0, cat='Continuous') for t in range(time_horizon)]\n Firings = [pulp.LpVariable(f"Firings_{t}", lowBound=0, cat='Continuous') for t in range(time_horizon)]\n Overtime = [pulp.LpVariable(f"Overtime_{t}", lowBound=0, cat='Continuous') for t in range(time_horizon)]\n\n # Objective Function\n profit_term = pulp.lpSum(profit_per_unit[i] * Amount_sold[i][t] for i in range(products_total) for t in range(time_horizon))\n holding_cost_term = pulp.lpSum(holding_cost_per_unit[i] * Inventory[i][t] for i in range(products_total) for t in range(time_horizon))\n wage_cost_term = pulp.lpSum(hourly_wage * Workforce[t] for t in range(time_horizon))\n overtime_cost_term = pulp.lpSum(hourly_wage_overtime * Overtime[t] for t in range(time_horizon))\n hiring_cost_term = pulp.lpSum(hiring_cost * Hires[t] for t in range(time_horizon))\n layoff_cost_term = pulp.lpSum(layoff_cost * Firings[t] for t in range(time_horizon))\n\n # Adding a non-linear profit term to simulate economies of scale\n scale_factor = 0.95 # Simulating economies of scale where more production reduces cost per unit\n scale_profit_term = pulp.lpSum((profit_per_unit[i] * Amount_produced[i][t] - (production_time[i][t]**scale_factor)) for i in range(products_total) for t in range(time_horizon))\n\n problem += profit_term + scale_profit_term - holding_cost_term - wage_cost_term - overtime_cost_term - hiring_cost_term - layoff_cost_term\n\n # Constraints\n for i in range(products_total):\n for t in range(time_horizon):\n problem += Amount_sold[i][t] >= min_sales[i][t], f"Min_Sales_{i}{t}"\n problem += Amount_sold[i][t] <= max_demand[i][t], f"Max_Demand{i}{t}"\n if t == 0:\n problem += Inventory[i][t] == initial_inventory[i] + Amount_produced[i][t] - Amount_sold[i][t], f"Inventory_Balance{i}{t}"\n else:\n problem += Inventory[i][t] == Inventory[i][t-1] + Amount_produced[i][t] - Amount_sold[i][t], f"Inventory_Balance{i}{t}"\n\n for j in range(workstations_total):\n for t in range(time_horizon):\n problem += pulp.lpSum(production_time[i][j] * Amount_produced[i][t] for i in range(products_total)) <= capacity[j][t] + Overtime[t], f"Capacity{j}{t}"\n\n # Adding new strategy: Limit hiring and firing in periods to reduce costs\n min_hiring_period = [1, 1, 0, 0, 0, 1]\n min_firing_period = [0, 1, 0, 0, 1, 0]\n\n for t in range(time_horizon):\n if t == 0:\n problem += Workforce[t] == initial_workers[0] + Hires[t] - Firings[t], f"Workforce_Balance{t}"\n else:\n problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n \n problem += pulp.lpSum(worker_hours_per_product * Amount_produced[i][t] for i in range(products_total)) <= Workforce[t] + Overtime[t], f"Work_Ontime_Constraint_{t}"\n \n # Apply min hiring and firing periods\n if min_hiring_period[t] == 1:\n problem += Hires[t] <= np.min(initial_workers), f"Min_Hiring_Period_{t}"\n if min_firing_period[t] == 1:\n problem += Firings[t] <= np.min(initial_workers), f"Min_Firing_Period_{t}"\n \n # Solve the problem\n problem.solve()\n\n # Output the results\n decision_variables = {\n "Produced": [[Amount_produced[i][t].varValue for t in range(time_horizon)] for i in range(products_total)],\n "Sale": [[Amount_sold[i][t].varValue for t in range(time_horizon)] for i in range(products_total)],\n "Stock": [[Inventory[i][t].varValue for t in range(time_horizon)] for i in range(products_total)],\n "Workforce": [Workforce[t].varValue for t in range(time_horizon)],\n "Hires": [Hires[t].varValue for t in range(time_horizon)],\n "Firings": [Firings[t].varValue for t in range(time_horizon)],\n "Overtime": [Overtime[t].varValue for t in range(time_horizon)]\n }\n profit = pulp.value(problem.objective)\n\n for key, value in decision_variables.items():\n print(f'{key}: {value}')\n\n print(f'Profit Earned: {profit}')\n \n return decision_variables, profit\n\nminlp_optimization_model(time_horizon, products_total, workstations_total, max_demand, min_sales, \n production_time, capacity, profit_per_unit, holding_cost_per_unit, initial_inventory, \n resource_requirement, resource_capacity, back_orders_penalty, \n worker_hours_per_product, hourly_wage, hourly_wage_overtime, hiring_cost, layoff_cost, initial_workers)\n \n\n### Table: New Strategies\n| Strategy | Description | Reason for Implementation |\n| --- | --- | --- |\n| Limit Hiring/Firing Period | Restrict hiring and firing to certain periods to stabilize the workforce and reduce costs associated with frequent changes. | Based on the performance feedback, frequent workforce changes were leading to high costs. This strategy aims to stabilize the workforce over the periods. |\n\n### Real Life Implications and Interpretation\nBy introducing workforce stabilization strategies, the manufacturing system will see fewer disruptions caused by constant hiring and firing. This approach would lead to better prediction and planning capabilities, leading to potentially smoother operations and less time-consuming workforce training and onboarding processes. In essence, the factory's workforce would become more stable and efficient, leading to a more predictable and potentially more productive environment."""

LP Iterate

In [ ]:
import concurrent.futures
import replicate
import time
import sys
import io
import re
import ast

ModuleNotFoundError: No module named 'replicate'

LP Model

In [ ]:
def chanceLP(ps, pf, explain_example, number_of_executions=2):
    previous_solution = ps
    previous_feedback = pf

    counter = 0
    seed = 42
    max_retry_attempts = 3

    while counter < number_of_executions:
        input = {
            # "prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, at the end of the code don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new storage strategies, production, and sales strategies (and workforce strategies if such elements appear in the current LP model) based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned and such strategies exist already, still add at least one more. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX and so on ..... If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function, or for workforce): [description strategy], reason for this: [what problem detected in the performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!]. Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out only one additional extension compared to the LP model that I've shown you.",
            "prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, at the end of the code don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new storage strategies, production, and sales strategies (and workforce strategies if such elements appear in the current model) based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned and such strategies exist already, still add at least one more. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX and so on ..... If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function, or for workforce), description strategy, and reason for this: (= which problem(s) detected in the last performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!). Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out at least one additional extension/new strategy compared to the LP model that I've shown you as the current solution.",
            "max_new_tokens": 2500
        }

        client = replicate.Client(api_token=os.getenv("REPLICATE_API_TOKEN"))

        attempt = 1
        while attempt <= max_retry_attempts:
            try:
                # Make the prediction
                prediction = client.run(
                    "meta/meta-llama-3-70b-instruct",
                    input=input
                )

                secondary_solution = "".join(prediction)
                print(secondary_solution)

                # Extracting the python code
                py_code_s = extract_python_code_manual(secondary_solution)

                # Redirect stdout to capture prints
                old_stdout = sys.stdout  # Save the old state
                new_stdout = io.StringIO()
                sys.stdout = new_stdout

                # Execute the code
                exec(py_code_s[0])

                # Restore stdout to its original state
                sys.stdout = old_stdout

                break  # Exit the retry loop if successful
            except Exception as e:
                print(f'Failed attempt {attempt}: {e}')
                attempt += 1

        # If the maximum number of retry attempts is reached, return the current winner version
        if attempt > max_retry_attempts:
            print("Max retry attempts reached. Returning the current winner version.")
            return previous_solution

        py_code_p = extract_python_code_manual(previous_solution)

        # Get the output and work with it
        execs = new_stdout.getvalue()

        # Redirect stdout to capture prints
        old_stdout = sys.stdout  # Save the old state
        new_stdout = io.StringIO()
        sys.stdout = new_stdout
        # Execute the code
        exec(py_code_p[0])
        # Restore stdout to its original state
        sys.stdout = old_stdout
        # Get the output and work with it
        execp = new_stdout.getvalue()

        # Define regex patterns for each matrix
        patterns = {
            "Produced": r"Produced: (\[\[.*?\]\])",
            "Sale": r"Sale: (\[\[.*?\]\])",
            "Stock": r"Stock: (\[\[.*?\]\])",
            "Overtime": r"Overtime: (\[.*?\])",
            "Workforce": r"Workforce: (\[.*?\])",
            "Hires": r"Hires: (\[.*?\])",
            "Firings": r"Firings: (\[.*?\])",
            "Profit": r"Profit: (.*?)\n"
        }

        # Dictionary to hold the extracted matrices
        matrices = {}

        # Extract and evaluate each matrix
        for key, pattern in patterns.items():
            match = re.search(pattern, execp, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        # Access the matrices
        produced_Prev = matrices.get("Produced")
        sale_Prev = matrices.get("Sale")
        stock_Prev = matrices.get("Stock")

        overtime_Prev = matrices.get("Overtime")
        workers_Prev = matrices.get("Workforce")
        hired_Prev = matrices.get("Hires")
        fired_Prev = matrices.get("Firings")


        # Define regex patterns for each matrix
        patterns = {
            "Produced": r"Produced: (\[\[.*?\]\])",
            "Sale": r"Sale: (\[\[.*?\]\])",
            "Stock": r"Stock: (\[\[.*?\]\])",
            "Overtime": r"Overtime: (\[.*?\])",
            "Workforce": r"Workforce: (\[.*?\])",
            "Hires": r"Hires: (\[.*?\])",
            "Firings": r"Firings: (\[.*?\])",
            "Profit": r"Profit: (.*?)\n"
        }

        # Dictionary to hold the extracted matrices
        matrices = {}

        # Extract and evaluate each matrix
        for key, pattern in patterns.items():
            match = re.search(pattern, execs, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        # Access the matrices
        produced_Second = matrices.get("Produced")
        sale_Second = matrices.get("Sale")
        stock_Second = matrices.get("Stock")
        overtime_Second = matrices.get("Overtime")
        workers_Second = matrices.get("Workforce")
        hired_Second = matrices.get("Hires")
        fired_Second = matrices.get("Firings")

        result_prev = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, seed)[-1]
        result_second = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, seed)[-1]

        result_prev_fb = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, seed)[:-1]
        result_second_fb = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, seed)[:-1]

        winner_profit, winner_input, winner_fb = compare_total_profits(result_prev, result_second, previous_solution, secondary_solution, result_prev_fb, result_second_fb)
        print("Higher Total Profit:", winner_profit)
        print("Winner Input:", winner_input)
        print("Winner Feedback:", winner_fb)

        previous_solution = winner_input
        previous_feedback = winner_fb

        # Example operation
        print(f"Execution {counter + 1}")

        # Increment the counter at the end of each loop iteration
        counter += 1
        seed += 1

    return winner_input


In [ ]:
# Define a function to run parallel tasks, before, the setting was 5/3
def run_parallel_tasks(ps, pf, explain_example, number_of_iterations=1, number_of_executions=5):
    collect_array = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(chanceLP, ps, pf, explain_example, number_of_executions) for _ in range(number_of_iterations)]
        for future in concurrent.futures.as_completed(futures):
            try:
                collect_array.append(future.result())
            except Exception as e:
                print(f"An error occurred: {e}")
    return collect_array

In [ ]:
collect_array = run_parallel_tasks(previous_solution_bu, previous_feedback_bu, explain_example)
print(collect_array)



Here is a modified version of the code that includes new storage, production, and sales strategies, as well as a new workforce strategy. The new strategies are designed to address the problems detected in the simulation feedback.

```python
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Define the function
def optimize_production():
    # Input data
    time_horizon = 6
    products_total = 4
    workstations_total = 4
    ressources_total = 4

    profit = [132.0, 813.0, 225.0, 131.0]
    holding_costs = [13.0, 18.0, 15.0, 13.0]

    min_sales = [
        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],
        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],
        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],
    ]

    max_demand = [
        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],
        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],
        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],
        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],
    ]

    cap

In [ ]:
collect_array

['\n\nHere is a revised version of the code that incorporates new strategies for storage, production, sales, and workforce:\n```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n    ressources_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [42

In [ ]:
# Install the replicate library
!pip install replicate
import replicate

# Import necessary libraries
from getpass import getpass
import os
import replicate

# Prompt for the API token
REPLICATE_API_TOKEN = getpass("Enter your Replicate API token: ")
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

# Verify the token is set correctly
print("API Token set:", os.getenv("REPLICATE_API_TOKEN"))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.7 MB/s eta 0:00:00
Enter your Replicate API token: ··········
API Token set: r8_QZSwIBTXbnKVj7S5FAAo92nDI8pWz4e0d3fDM


LP MODEL

In [ ]:
previous_solution = """```python
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Define the function
def optimize_production():
    # Input data
    time_horizon = 6
    products_total = 4
    workstations_total = 4
    ressources_total = 4

    profit = [132.0, 813.0, 225.0, 131.0]
    holding_costs = [13.0, 18.0, 15.0, 13.0]

    min_sales = [
        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],
        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],
        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],
    ]

    max_demand = [
        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],
        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],
        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],
        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],
    ]

    capacity = [
        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],
        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],
        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],
        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],
    ]

    production_time = [
        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],
        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],
        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],
        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],
    ]

    initial_inventory = [41.0, 58.0, 31.0, 38.0]
    yield_loss = [
        [0.9, 0.7, 0.9, 0.8],
        [0.8, 0.6, 0.95, 0.8],
        [0.9, 0.75, 0.9, 0.78],
        [0.95, 0.87, 0.97, 0.98],
    ]
    capacity_reduction_factor = 0.9

    # New data required for the function
    worker_hours_per_product = 2.0
    hourly_wage = 20.0
    hourly_wage_overtime = 35.0
    hiring_cost = 10.0
    layoff_cost = 5.0
    initial_workers = 10

    # Create the problem
    problem = LpProblem("Advanced_Supply_Chain_Optimization", LpMaximize)

    # Index ranges for products, workstations, and periods
    products = range(products_total)
    workstations = range(workstations_total)
    periods = range(time_horizon)

    # Decision Variables
    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]
    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]
    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]
    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]
    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]
    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]
    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]

    # Objective Function
    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)
    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)
    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)
    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)
    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)
    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)
    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term

    # Constraints
    for i in products:
        for t in periods:
            # Sales constraints
            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"
            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"
            # Inventory balance constraint
            if t == 0:
                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"
            else:
                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"

    for j in workstations:
        for t in periods:
            # Adjusted capacity constraint
            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <= \
                       capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"

    for t in periods:
        # Workforce constraints
        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"
        if t == 0:
            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"
        else:
            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"

    # Solve the problem
    problem.solve()

    # Output the results
    print_result = lambda s, v: [f"{s}: {[[v[i][t].varValue for t in periods] for i in products]}" if isinstance(v[0], list) else f"{s}: {[v[t].varValue for t in periods]}" for v in [v]]

    # Extract the optimization results
    results = {
        "Produced": [[Produced[i][t].varValue for t in periods] for i in products],
        "Sale": [[Sale[i][t].varValue for t in periods] for i in products],
        "Stock": [[Stock[i][t].varValue for t in periods] for i in products],
        "Workforce": [Workforce[t].varValue for t in periods],
        "Hires": [Hires[t].varValue for t in periods],
        "Firings": [Firings[t].varValue for t in periods],
        "Overtime": [Overtime[t].varValue for t in periods],
        "Profit": pulp.value(problem.objective)
    }

    for key, value in results.items():
        print(f"{key}: {value}")

    return results

# Call the function to optimize production
optimize_production()
```"""

MINLP

In [ ]:
previous_solution = '''```python
import pulp

# Input data
time_horizon = 6
products_total = 4
workstations_total = 4
ressources_total = 4

min_sales = [
    [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
    [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],
    [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],
    [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],
]
max_demand = [
    [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],
    [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],
    [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],
    [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],
]
capacity = [
    [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],
    [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],
    [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],
    [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],
]
production_time = [
    [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],
    [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],
    [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],
    [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],
]
resource_requirement = [
    [300.8, 30.5, 1.0, 3.0],
    [5.5, 14.3, 33.0, 10.0],
    [50.0, 14.0, 2.0, 40.0],
    [2.0, 6.0, 40.0, 1.0],
]
resource_capacity = [
    [100.0, 8345.0, 1417.0, 407.0, 20.0, 290.0],
    [25.0, 41.0, 338.0, 210.0, 405.0, 2090.0],
    [7103.0, 30.0, 2007.0, 430.0, 90.0, 140.0],
    [200.0, 3001.0, 470.0, 2109.0, 140.0, 203.0],
]
initial_inventory = [41.0, 58.0, 31.0, 38.0]
back_orders_penalty = [5, 6, 7, 2]
profit_per_unit = [132.0, 83.0, 225.0, 131.0]
holding_cost_per_unit = [3.0, 8.0, 5.0, 1.0]


worker_hours_per_product = 2.0
hourly_wage = 20.0
hourly_wage_overtime = 35.0
hiring_cost = 100.0
layoff_cost = 50.0
initial_workers = [50, 60, 70, 80]

def optimization_model(time_horizon, products_total, workstations_total, max_demand, min_sales,
                       production_time, capacity, profit_per_unit, holding_cost_per_unit, initial_inventory,
                       resource_requirement, resource_capacity, back_orders_penalty,
                       worker_hours_per_product, hourly_wage, hourly_wage_overtime, hiring_cost, layoff_cost, initial_workers):

    # Create the problem
    problem = pulp.LpProblem("Advanced_Supply_Chain_Optimization", pulp.LpMaximize)

    # Decision Variables
    Amount_produced = [[pulp.LpVariable(f"Produced_{i}_{t}", lowBound=0, cat='Continuous') for t in range(time_horizon)] for i in range(products_total)]
    Amount_sold = [[pulp.LpVariable(f"Sale_{i}_{t}", lowBound=0, cat='Continuous') for t in range(time_horizon)] for i in range(products_total)]
    Inventory = [[pulp.LpVariable(f"Stock_{i}_{t}", lowBound=0, cat='Continuous') for t in range(time_horizon)] for i in range(products_total)]
    Workforce = [pulp.LpVariable(f"Workforce_{t}", lowBound=0, cat='Continuous') for t in range(time_horizon)]
    Hires = [pulp.LpVariable(f"Hires_{t}", lowBound=0, cat='Continuous') for t in range(time_horizon)]
    Firings = [pulp.LpVariable(f"Firings_{t}", lowBound=0, cat='Continuous') for t in range(time_horizon)]
    Overtime = [pulp.LpVariable(f"Overtime_{t}", lowBound=0, cat='Continuous') for t in range(time_horizon)]

    # Objective Function
    profit_term = pulp.lpSum(profit_per_unit[i] * Amount_sold[i][t] for i in range(products_total) for t in range(time_horizon))
    holding_cost_term = pulp.lpSum(holding_cost_per_unit[i] * Inventory[i][t] for i in range(products_total) for t in range(time_horizon))
    wage_cost_term = pulp.lpSum(hourly_wage * Workforce[t] for t in range(time_horizon))
    overtime_cost_term = pulp.lpSum(hourly_wage_overtime * Overtime[t] for t in range(time_horizon))
    hiring_cost_term = pulp.lpSum(hiring_cost * Hires[t] for t in range(time_horizon))
    layoff_cost_term = pulp.lpSum(layoff_cost * Firings[t] for t in range(time_horizon))
    problem += profit_term - holding_cost_term - wage_cost_term - overtime_cost_term - hiring_cost_term - layoff_cost_term

    # Constraints
    for i in range(products_total):
        for t in range(time_horizon):
            problem += Amount_sold[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"
            problem += Amount_sold[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"
            if t == 0:
                problem += Inventory[i][t] == initial_inventory[i] + Amount_produced[i][t] - Amount_sold[i][t], f"Inventory_Balance_{i}_{t}"
            else:
                problem += Inventory[i][t] == Inventory[i][t-1] + Amount_produced[i][t] - Amount_sold[i][t], f"Inventory_Balance_{i}_{t}"

    for j in range(workstations_total):
        for t in range(time_horizon):
            problem += pulp.lpSum(production_time[i][j] * Amount_produced[i][t] for i in range(products_total)) <= capacity[j][t] + Overtime[t], f"Capacity_{j}_{t}"

    for t in range(time_horizon):
        if t == 0:
            problem += Workforce[t] == initial_workers[0] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"
        else:
            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"
        problem += pulp.lpSum(worker_hours_per_product * Amount_produced[i][t] for i in range(products_total)) <= Workforce[t] + Overtime[t], f"Work_Ontime_Constraint_{t}"

    # Solve the problem
    problem.solve()

    # Output the results
    decision_variables = {
        "Produced": [[Amount_produced[i][t].varValue for t in range(time_horizon)] for i in range(products_total)],
        "Sale": [[Amount_sold[i][t].varValue for t in range(time_horizon)] for i in range(products_total)],
        "Stock": [[Inventory[i][t].varValue for t in range(time_horizon)] for i in range(products_total)],
        "Workforce": [Workforce[t].varValue for t in range(time_horizon)],
        "Hires": [Hires[t].varValue for t in range(time_horizon)],
        "Firings": [Firings[t].varValue for t in range(time_horizon)],
        "Overtime": [Overtime[t].varValue for t in range(time_horizon)]
    }
    profit = pulp.value(problem.objective)

    for key, value in decision_variables.items():
        print(f'{key}: {value}')

    print(f'Profit Earned: {profit}')

    return decision_variables, profit

optimization_model(time_horizon, products_total, workstations_total, max_demand, min_sales,
                   production_time, capacity, profit_per_unit, holding_cost_per_unit, initial_inventory,
                   resource_requirement, resource_capacity, back_orders_penalty,
                   worker_hours_per_product, hourly_wage, hourly_wage_overtime, hiring_cost, layoff_cost, initial_workers)
```'''